In [38]:
import requests
import unicodedata

In [39]:
cnpj = '15436220000130'
email = 'recifeplacas@hotmail.com'
 # URL api Brasil para cnpj
def pegar_dados_empresa(cnpj):
    requisicao = requests.get(f'https://brasilapi.com.br/api/cnpj/v1/{cnpj}')
    dados_empresa = requisicao.json()
    return dados_empresa

In [40]:
dados_empresa = pegar_dados_empresa(cnpj)
print(dados_empresa)

{'uf': 'PE', 'cep': '50690130', 'qsa': [{'pais': None, 'nome_socio': 'SEVERINO CRISTOVAO DA SILVA FILHO', 'codigo_pais': None, 'faixa_etaria': 'Entre 61 a 70 anos', 'cnpj_cpf_do_socio': '***255494**', 'qualificacao_socio': 'Sócio-Administrador', 'codigo_faixa_etaria': 7, 'data_entrada_sociedade': '2012-10-02', 'identificador_de_socio': 2, 'cpf_representante_legal': '***000000**', 'nome_representante_legal': '', 'codigo_qualificacao_socio': 49, 'qualificacao_representante_legal': 'Não informada', 'codigo_qualificacao_representante_legal': 0}], 'cnpj': '15436220000130', 'pais': None, 'email': None, 'porte': 'MICRO EMPRESA', 'bairro': 'IPUTINGA', 'numero': '32', 'ddd_fax': '', 'municipio': 'RECIFE', 'logradouro': 'MAJOR MARCELO MENEZES', 'cnae_fiscal': 3299003, 'codigo_pais': None, 'complemento': 'CASA', 'codigo_porte': 1, 'razao_social': 'CRISTOVAO E MEDEIROS INDUSTRIA E COMERCIO DE PLACAS LTDA', 'nome_fantasia': 'RECIFE PLACAS', 'capital_social': 105000, 'ddd_telefone_1': '8188668020', 

In [41]:
cidade_empresa = dados_empresa.get('municipio')
cidade_empresa

'RECIFE'

In [42]:
estado_empresa = dados_empresa.get('uf')
estado_empresa

'PE'

In [43]:
def remover_acentos(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )

def pegar_codigo_cidade(estado, cidade):
    
    # URL api Brasil para ibge
    url = f'https://brasilapi.com.br/api/ibge/municipios/v1/{estado}'
    requisicao = requests.get(url)
    dados_empresa = requisicao.json()
    for dicionario in dados_empresa:
        if cidade == remover_acentos(dicionario.get('nome')):
            cidade = dicionario.get('nome'), 
            codigo_cidade = dicionario.get('codigo_ibge')
    return cidade[0], codigo_cidade

In [44]:
pegar_codigo_cidade(estado_empresa, cidade_empresa)


('RECIFE', '2611606')

In [45]:
print(dados_empresa.get('email'))

None


In [52]:
url = 'https://api.sandbox.plugnotas.com.br/empresa'
headers = {
    'X-API-KEY': '2da392a6-79d2-4304-a8b7-959572c7e44d'
}

body = {
    'cpfCnpj': dados_empresa.get('cnpj'),
    
    # Inscriçao Estadual tem que ver um jeito automatico ou forca usuario digitar
    'inscricaoEstadual': '048411159',
    'razaoSocial': dados_empresa.get('razao_social'),
    
    # certificado digital se cadatra antes pega o ID e assim pode cadastra empresa 
    'certificado': '67829b65b931bb7d670c7341', #'ID do certificado a ser vinculado a empresa. Obtido através da rota /certificado',
    'simplesNacional': dados_empresa.get('opcao_pelo_simples'),
    
    # O regime tributario tera que ver com o contador da empresa cadastrada
    'regimeTributario': 1,
    'regimeTributarioEspecial': 6,
    
    'endereco': {
        'bairro': dados_empresa.get('bairro'),
        'cep': dados_empresa.get('cep'),
        
        # criar funcao para bucar codigo da cidade do IBGE
        'codigoCidade': pegar_codigo_cidade(estado_empresa, cidade_empresa)[1],
        'estado': dados_empresa.get('uf'),
        'logradouro': dados_empresa.get('logradouro'),
        'numero': dados_empresa.get('numero'),
        'tipoLogradouro': dados_empresa.get('descricao_tipo_de_logradouro'),        
    },
    # o email sera na hora do cadastro o email do usuario ja logado
    'email': email,
    
    'nfe': {
        'ativo': True,
        'tipoContrato': 1
    },
}

requisicao = requests.post(url, headers=headers, json=body)
requisicao.status_code

409

In [53]:
requisicao.json()

{'error': {'message': 'Já existe uma Empresa com os parâmetros informados',
  'data': {'new': {'cnpj': '15436220000130'},
   'current': {'cnpj': '15436220000130',
    'razaoSocial': 'CRISTOVAO E MEDEIROS INDUSTRIA E COMERCIO DE PLACAS LTDA'}}}}